In [73]:
import numpy as np
import os
import pandas as pd
import csv
import re
from collections import Counter

In [127]:
def load_data(fPath):
    data = []
    maxNumber = 0
    with open(fPath, newline='') as csvfile:
        dataFile = csv.reader(csvfile, delimiter=',', skipinitialspace=True)
        for row in dataFile:
            curMax = int(max(row))
            maxNumber = curMax if curMax > maxNumber else maxNumber
            data.append(row)
    
    return data, maxNumber

def load_params(fPath):
    params = []
    with open(fPath, newline='') as csvfile:
        dataFile = csv.reader(csvfile, delimiter=',', skipinitialspace=True)
        # Extract data in tuples. Split line delimited by '=' then find if the left term has a number key or refers to rest or SDC terms.
        for row in dataFile:
            key, val = row[0].split('=')            
            key = re.findall(r'\d+', key)
            key = int(key[0]) if key  != [] else ('rest' if 'rest' in row[0] else 'SDC')           
            params.append((key,float(val)))
    print(params)
    # format params into a sorted array
    return params

def create_M(data, rawParams):
    uniqueItems = []
    # Concat all list in a big one, so we can find the distinct items
    for row in data:
        uniqueItems += row
    uniqueItems = np.sort(np.array(np.unique(uniqueItems), dtype=int))
    #print(uniqueItems)
    # Initialize param file. Column 0 has the items, columns 1 the MIS stated in 'rest' for init
    params = np.ones((2,len(uniqueItems)))
    params[0,:] = uniqueItems; params[1,:] = rawParams[-2][1]
    #print(params, rawParams[-2][1])
    # Find which items are given in the raw Params file. Last 2 lines are always rest and SDC
    for i in rawParams[:-2]:
        item, sup = i[0],i[1]
        params[1, np.where(params[0,:] == item)] = sup   
    # Now sort the params array to create M
    indices = np.argsort(params[1,:])
    M = np.array([params[0, indices], params[1,indices]])
    print('M: ', M)
    return M, rawParams[-1][1]

dataPath = "dummyData1.txt"
paramsPath = "dummyParams1.txt"
T, maxNumber = load_data(dataPath)
rawParams = load_params(paramsPath)
M, SDC = create_M(T, rawParams)

[(1, 0.02), (2, 0.04), ('rest', 0.01), ('SDC', 0.003)]
M:  [[3.e+00 4.e+00 5.e+00 6.e+00 7.e+00 8.e+00 9.e+00 1.e+01 2.e+01 3.e+01
  4.e+01 5.e+01 1.e+00 2.e+00]
 [1.e-02 1.e-02 1.e-02 1.e-02 1.e-02 1.e-02 1.e-02 1.e-02 1.e-02 1.e-02
  1.e-02 1.e-02 2.e-02 4.e-02]]


In [128]:
def init_pass(M,T):
    
    return 1
print(T)

[['10', '50', '30', '40'], ['1', '2', '4', '6', '8', '9', '10'], ['3', '4', '5'], ['7', '9', '20']]
